## 7. Asyncronous Programming

### async and await keywords
Use always async/await methods when there's an inbound/outbound process

In [11]:
Console.WriteLine($"[{DateTime.Now}] Waiting 1s.");
await Task.Delay(1000);
Console.WriteLine($"[{DateTime.Now}] Task completed.");

[7/17/2025 12:56:24 PM] Waiting 1s.
[7/17/2025 12:56:25 PM] Task completed.


### Concurrent execution
You can run multiple Tasks at once using methods like Task.WhenAll, which waits all the Tasks to be completed

In [12]:

Console.WriteLine($"[{DateTime.Now}] Running concurrent tasks.");
await Task.WhenAll(
    Task.Delay(1000),
    Task.Delay(2000) // As this task takes longer, WhenAll will complete after 2s
);
Console.WriteLine($"[{DateTime.Now}] Tasks completed.");

[7/17/2025 12:56:25 PM] Running concurrent tasks.
[7/17/2025 12:56:27 PM] Tasks completed.


### Cancellation tokens
You can abort a task using cancellation tokens. When building async code, always try to accept `CancellationToken` in the signature.

Let's review the following example:
1. We use a `CancellationTokenSource` to build a `CancellationToken`.
2. We run 2 tasks in parallel using the `Task.WhenAll` method.
2. `CancelTokenAfter1s` triggers a cancellation token after 1s delay on the `CancellationTokenSource` instance.
2. `DoWorkAsync` method waits 2s to complete. Receives the cancellation token controlled by previous task. It will never finish since the task will be cancelled after 1s.
3. After 1s, the `DoWorkAsync` execution is aborted
4. The Task.WhenAll task is completed

In [14]:
using System.Threading;

public class AsyncExample
{
    public async Task DoWorkAsync(CancellationToken cancellationToken)
    {
        try 
        {
            Console.WriteLine($"[{DateTime.Now}] DoWorkAsync - Starting work.");
            await Task.Delay(2000, cancellationToken);
            Console.WriteLine($"[{DateTime.Now}] DoWorkAsync - Work completed.");
        } 
        catch (TaskCanceledException) 
        {
            Console.WriteLine($"[{DateTime.Now}] DoWorkAsync - Work was cancelled.");
        }
    }

    public async Task CancelTokenAfter1s(CancellationTokenSource cts) 
    {
        await Task.Delay(1000);
        Console.WriteLine($"[{DateTime.Now}] CancelTokenAfter1s - Cancelling work.");
        cts.Cancel();
    }
}

var cts = new CancellationTokenSource();
var example = new AsyncExample();

// Run both tasks concurrently
await Task.WhenAll(
    example.CancelTokenAfter1s(cts),
    example.DoWorkAsync(cts.Token) // This task will be cancelled
);

[7/17/2025 12:56:27 PM] DoWorkAsync - Starting work.
[7/17/2025 12:56:28 PM] CancelTokenAfter1s - Cancelling work.
[7/17/2025 12:56:28 PM] DoWorkAsync - Work was cancelled.


### Fire and forget
Sometimes you may need to run a task in the background without waiting the result. This is known as a fire and forget task.

This is specially **dangerous** as it's hard to monitor when a task is completed or if it resulted with errors. Avoid this as much as possible.

In [13]:
Console.WriteLine($"[{DateTime.Now}] First message.");
Task.Delay(1000); // This may produce warnings if not awaited
Console.WriteLine($"[{DateTime.Now}] Second message.");

[7/17/2025 12:56:27 PM] First message.
[7/17/2025 12:56:27 PM] Second message.


### Synchronous processing
You can ran an async task in a synchronous way. This is **dangerous** and can create deadlocks or performance problems, but sometimes is useful when building supporting code for tests, etc.

In [15]:
Console.WriteLine($"[{DateTime.Now}] Waiting 1s.");
Task.Delay(1000).GetAwaiter().GetResult(); // Synchronous wait, dangerous!
Console.WriteLine($"[{DateTime.Now}] Task completed.");

[7/17/2025 12:59:13 PM] Waiting 1s.
[7/17/2025 12:59:14 PM] Task completed.
